In [1]:
import os
import numpy as np
import tensorflow as tf
import random
import pandas as pd
import math
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [78]:
patinfo['Disease'] , columns = pd.factorize(patinfo.Disease)
print columns

train, test = train_test_split(patinfo, test_size=0.3, random_state=0, stratify=patinfo[['Disease']])
print train.shape
print test.shape
#print train['Disease'].value_counts()
#print test['Disease'].value_counts()

Index([u'Myocardial infarction', u'Other'], dtype='object')
(91, 49)
(40, 49)


In [79]:
xtrain = []
ytrain = []
for k, r in train.iterrows():
    print r['patientID']
    data = pd.read_csv("./TrainECGData/"+str(r['patientID'])+"/0.csv")
    data.drop(["'sample #'"], axis = 1)
    #xtrain.append(data.iloc[:2000].values)
    mini = data.iloc[:2000]
    xtrain.append(np.reshape(mini.values, (1,np.product(mini.shape)))[0])
    
    lab = np.zeros(2)
    lab[r['Disease']] = 1
    ytrain.append(lab)

xtrain = np.array(xtrain)
print xtrain.shape
ytrain = np.array(ytrain)
print ytrain.shape
'''
data1 = pd.read_csv("./TrainECGData/0/0.csv")
data1 = data1.drop(["'sample #'"], axis = 1)
#data1['label'] = np.zeros(data1.shape[0])
data2 = pd.read_csv("./TrainECGData/22/0.csv")
data2 = data2.drop(["'sample #'"], axis = 1)
#data2['label'] = np.ones(data2.shape[0])
print data1.shape, data2.shape
data = pd.concat([data1.iloc[:2000],data2.iloc[:2000]])

print data.shape
'''


51
178
65
194
61
103
111
115
292
175
11
278
113
277
81
155
48
275
74
258
247
8
29
154
291
153
162
182
152
223
216
265
208
266
91
62
173
84
77
141
239
217
241
86
44
49
158
199
284
242
172
129
146
183
230
112
59
108
231
42
89
101
7
221
279
206
13
78
37
0
280
12
282
201
3
33
159
6
124
176
207
23
26
114
47
16
186
73
75
271
46
(91, 64000)
(91, 2)


'\ndata1 = pd.read_csv("./TrainECGData/0/0.csv")\ndata1 = data1.drop(["\'sample #\'"], axis = 1)\n#data1[\'label\'] = np.zeros(data1.shape[0])\ndata2 = pd.read_csv("./TrainECGData/22/0.csv")\ndata2 = data2.drop(["\'sample #\'"], axis = 1)\n#data2[\'label\'] = np.ones(data2.shape[0])\nprint data1.shape, data2.shape\ndata = pd.concat([data1.iloc[:2000],data2.iloc[:2000]])\n\nprint data.shape\n'

In [81]:
tf.reset_default_graph()

learning_rate = tf.placeholder(tf.float32, shape=[])


x = tf.placeholder(tf.float32, [None, xtrain.shape[1]])

y = tf.placeholder(tf.float32, [None, ytrain.shape[1]])

w1 = tf.get_variable("W1", shape=[xtrain.shape[1], 5000], initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.get_variable("b1", shape=[5000], initializer=tf.contrib.layers.xavier_initializer())

w2 = tf.get_variable("W2", shape=[5000, 1000], initializer=tf.contrib.layers.xavier_initializer())

b2 = tf.get_variable("b2", shape=[1000], initializer=tf.contrib.layers.xavier_initializer())

w3 = tf.get_variable("W3", shape=[1000, 1000], initializer=tf.contrib.layers.xavier_initializer())

b3 = tf.get_variable("b3", shape=[1000], initializer=tf.contrib.layers.xavier_initializer())

w4 = tf.get_variable("W4", shape=[1000, 200], initializer=tf.contrib.layers.xavier_initializer())

b4 = tf.get_variable("b4", shape=[200], initializer=tf.contrib.layers.xavier_initializer())

wO = tf.get_variable("WO", shape=[200, 9], initializer=tf.contrib.layers.xavier_initializer())

bO = tf.get_variable("bO", shape=[9], initializer=tf.contrib.layers.xavier_initializer())


H1 = tf.add(tf.matmul(x, w1) , b1)
H1 = tf.nn.relu(H1)
tf.nn.dropout(H1, .5)
H1 = tf.add(tf.matmul(H1, w2) , b2)
H1 = tf.nn.relu(H1)
tf.nn.dropout(H1, .5)
H1 = tf.add(tf.matmul(H1, w3) , b3)
H1 = tf.nn.relu(H1)
tf.nn.dropout(H1, .5)
H1 = tf.add(tf.matmul(H1, w4) , b4)
H1 = tf.nn.relu(H1)

out_layer = tf.add(tf.matmul(H1, wO) , bO)
testo= tf.nn.softmax(out_layer)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out_layer, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer()


In [82]:
epoch = 0
bsize = 16
with tf.Session() as sess:
    sess.run(init)
   
    saver = tf.train.Saver(tf.trainable_variables())
    #saver.restore(sess, './model.ckpt')
    
    while epoch < 20000:
        avg_cost = 0.
        ind = np.random.choice(xtrain.shape[0], bsize, replace=False)
        _, c = sess.run([optimizer, cost], feed_dict={x : xtrain[ind], y: ytrain[ind] , learning_rate : 0.0005})
        epoch += 1
        print c
        
        if epoch % 100 == 0:
            saver.save(sess, './model.ckpt')
            print '+============+ saving +===============++++++++++'

700.511
75716.2
5348.57
7209.73
15787.5
11340.6
62528.4
10.6019
22849.6
37796.3
15298.1
5858.12
358.908
28492.8
13882.4
5248.68
3505.34
2375.24
20964.9
8862.32
7848.88
4072.2
1936.01
3781.59
4340.88
4793.45
9324.1
64468.5
4720.94
6113.28
1798.5
3366.67
8426.39
8619.62
5207.1
6664.77
6365.84
6478.65
2946.67
1211.74
6210.31
340.461
1.2926
1618.03
5147.62
7167.43
1724.05
2483.95
8024.51
4124.97
11381.7
1604.31
333.263
6983.12
1230.57
103.917
4432.5
10361.2
5105.93
69.174
898.765
41.6177
1832.38
3652.18
405.357
24190.7
2587.33
1866.78
988.078
4957.37
2160.05
1963.5
6287.04
6829.87
4664.99
2278.03
7626.7
708.682
15660.1
2680.03
206.968
7214.73
1787.22
778.993
2719.87
1078.8
258.718
0.0
263.714
253.64
3302.18
2101.52
10718.7
3772.48
9535.22
16407.8
1724.0
4703.91
834.144
194.311
+============+ saving +===============++++++++++
920.896
857.758
4125.03
4501.94
2634.23
1614.44
43.0833
32.9667
858.793
3634.48
32.7351
1290.79
0.0
2717.38
1056.76
905.782
0.0
241.828
0.0
0.0
10181.9
3063.7
0.0
122.

KeyboardInterrupt: 

In [84]:
with tf.Session() as sess:
    sess.run(init)
   
    saver = tf.train.Saver(tf.trainable_variables())
    saver.restore(sess, "./model.ckpt")

    res = sess.run([testo],  feed_dict={x : xtest})#, xp: 
                                       #XTrainprov[ind],
                                       #np.zeros((len(ind), 2140)),
                                       
                                       #xtaxo: XTraintaxo[ind],
                                        #       y: YTRAIN[ind].toarray()})
    print res

INFO:tensorflow:Restoring parameters from ./model.ckpt
[array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]], dtype=float32)]


In [ ]:
y_true = []
for i in ytest:
    y_true.append(np.argmax(i))
y_pred = []
for i in res[0]:
    y_pred.append(np.argmax(i))
print len(y_pred)
print len(y_true)
from sklearn.metrics import f1_score
#y_true = [0, 1, 2, 0, 1, 2]
#y_pred = [0, 2, 1, 0, 0, 1]
f1_score(y_true, y_pred, average='macro') 

In [ ]:
##########################################################
#################### EDA & Rough work ###################
########################################################

In [92]:


patinfo = pd.read_csv('./Patient_Records_Train.csv')
print patinfo.shape
print patinfo.columns
patinfo = patinfo.sort_values('patientID')
patinfo = patinfo[(patinfo['Disease'] != 'Healthy control') & (patinfo['Disease'] != 'Unidentified')]
for t in ['Dysrhythmia', 'Valvular heart disease',
 'Cardiomyopathy', 'Bundle branch block', 'Hypertrophy', 'Myocarditis']:
    patinfo['Disease'] = patinfo['Disease'].str.replace(t, 'Other')
    
print patinfo['Disease'].unique()

(173, 49)
Index([u'patientID', u'Acute infarction (localization)',
       u'Additional diagnoses', u'Additional medication', u'Admission date',
       u'Aorta (at rest) diast', u'Aorta (at rest) mean',
       u'Aorta (at rest) syst', u'Cardiac index (at rest)',
       u'Cardiac index (load)', u'Cardiac output (at rest)',
       u'Cardiac output (load)', u'Catheterization date', u'Chest X-ray',
       u'Dosage (lytic agent)', u'ECG date', u'Echocardiography',
       u'Former infarction (localization)', u'In hospital medication',
       u'Infarction date', u'Infarction date (acute)',
       u'Left coronary artery stenoses (RCX)',
       u'Left coronary artery stenoses (RIVA)',
       u'Left ventricular enddiastolic pressure', u'Lytic agent',
       u'Medication after discharge', u'Medication pre admission',
       u'Number of coronary vessels involved',
       u'Peripheral blood Pressure diast', u'Peripheral blood Pressure syst',
       u'Previous infarction (1) date', u'Previous infarct

In [118]:
g1 = patinfo[patinfo['Disease'] == 'Myocardial infarction']
g2 = patinfo[patinfo['Disease'] == 'Other']

print np.intersect1d(g1['Acute infarction (localization)'],g2['Acute infarction (localization)'])
#### classify as Myocardial infarction if annything in this list
print np.setdiff1d(g1['Acute infarction (localization)'],g2['Acute infarction (localization)'])
## if NO, checking

print g1[g1['Acute infarction (localization)'] == ' no'][['Additional diagnoses', u'Additional medication']]

uniqueg1 = []
for t in g1['Additional diagnoses'].unique():
    uniqueg1.extend(t.split(', | .'))
uniqueg2 = []
for t in g2['Additional diagnoses'].unique():
    uniqueg2.extend(t.split(', | .'))

print np.intersect1d(uniqueg1, uniqueg2)

#if [' Arterial hypertension' ' no']:
#    check and go forward
#else
#np.setdiff1d(uniqueg1, uniqueg2)  #### if any in this, classify

print g1[g1['Additional diagnoses'].str.find('Arterial hypertension')]

#print np.intersect1d(g1['Additional diagnoses'],g2['Additional diagnoses'])
#print np.setdiff1d(g1['Additional diagnoses'],g2['Additional diagnoses'])

'''

groups = patinfo.groupby('Disease')
for index, group in groups:
    print index
    print 'Acute Infraction: ', group['Acute infarction (localization)'].unique()
    print 'Additional diagnosis', group['Additional diagnoses'].unique()
    print 'Additional medication', group['Additional medication'].unique()
'''

[' no']
[' anterior' ' antero-lateral' ' antero-septal' ' antero-septo-lateral'
 ' inferior' ' infero-latera' ' infero-lateral' ' infero-poster-lateral'
 ' infero-postero-lateral' ' posterior' ' postero-lateral']
                                  Additional diagnoses Additional medication
66                           Periperal atherosclerosis                   n/a
45    Recurrent ventricular fibrillation, Arterial ...                   n/a
11                                             unknown                   n/a
78    Palpitation, Postop. Posthetic valve replacem...                   n/a
37    Hypercholesterinemia, Recurrent ventricular t...                   n/a
80                 WPW, paroxysmal atrial fibrillation                   n/a
146   Recurrent ventricular tachycardia, Dilated ca...                   n/a
58    Dilated Cardiomyopathy secondary to coronary ...                   n/a
43      Recurrent ventricular tachycardia, ACVB (RIVA)                   n/a
133   Pericardial

KeyError: '[-1  1 -1 20 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 37 -1 -1 -1  1 -1 -1 -1\n -1 -1 -1 -1 20 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1  1  1  1  1\n -1 -1 41 -1 -1  1  1 32 -1 -1 -1 -1  1  1 -1 -1  1 -1 -1 80 -1 -1 -1 -1 -1\n  1 -1 -1 -1 -1 -1 73  1  1 -1 -1 -1 -1 -1 -1 -1  1 46 -1 -1  1 -1 -1] not in index'

In [80]:
xtest = []
ytest = []
for k, r in test.iterrows():
    print r['patientID']
    data = pd.read_csv("./TrainECGData/"+str(r['patientID'])+"/0.csv")
    data.drop(["'sample #'"], axis = 1)
    #xtrain.append(data.iloc[:2000].values)
    mini = data.iloc[:2000]
    xtest.append(np.reshape(mini.values, (1,np.product(mini.shape)))[0])
    
    lab = np.zeros(2)
    lab[r['Disease']] = 1
    ytest.append(lab)

xtest = np.array(xtest)
print xtest.shape
ytest = np.array(ytest)
print ytest.shape

140
262
17
127
79
128
193
209
191
188
174
215
69
228
288
109
283
289
55
273
76
22
52
95
286
53
256
232
34
54
97
27
205
36
259
83
219
102
130
120
(40, 64000)
(40, 2)
